## Tool Calling

In [ ]:
from typing import TypedDict, List, Any

from langchain_ollama import ChatOllama
from langchain.tools import tool
from langchain_core.messages import HumanMessage

from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
import os

os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langgraph-debug"

print("TRACING:", os.environ.get("LANGCHAIN_TRACING_V2"))
print("PROJECT:", os.environ.get("LANGCHAIN_PROJECT"))
print("API KEY SET:", bool(os.environ.get("LANGCHAIN_API_KEY")))

@tool
def duck() -> str:
    """Make a duck sound."""
    return "quack"


@tool
def dog() -> str:
    """Make a dog sound."""
    return "woff"

llm = ChatOllama(
    model="llama3.1:8B",
    temperature=0,
)

llm_with_tools = llm.bind_tools([duck, dog])

class State(TypedDict):
    user_text: str
    messages: List[Any]


def llm_node(state: State) -> dict:
    messages = state.get("messages", [])
    user_text = state.get("user_text")

    # First turn: inject HumanMessage exactly once
    if not messages:
        if user_text is None:
            raise ValueError("user_text missing")
        messages = [HumanMessage(content=user_text)]

    response = llm_with_tools.invoke(messages)

    return {
        # 🔑 APPEND — never replace
        "messages": messages + [response]
    }

tool_node = ToolNode([duck, dog])

graph = StateGraph(State)

graph.add_node("llm", llm_node)
graph.add_node("tools", tool_node)

graph.set_entry_point("llm")

# Decide whether to run tools or stop
graph.add_conditional_edges(
    "llm",
    tools_condition,
    {
        "tools": "tools",
        END: END
    }
)

# After tools run, we stop (simple example)
graph.add_edge("tools", END)

app = graph.compile()

result = app.invoke({
    "user_text": "I wonder what this animal with four leg says.",
    "messages": []
})


TRACING: true
PROJECT: langgraph-debug
API KEY SET: True


/var/folders/7j/qr6tvgr97s3b65k1tx46fry80000gn/T/ipykernel_21765/2253802776.py:38: ResourceWarning: unclosed <socket.socket fd=88, family=2, type=1, proto=6, laddr=('127.0.0.1', 50570), raddr=('127.0.0.1', 11434)>
  llm_with_tools = llm.bind_tools([duck, dog])


In [3]:
result

{'user_text': 'I wonder what this animal with four leg says.',
 'messages': [ToolMessage(content='woff', name='dog', tool_call_id='a0b852d5-e9a0-4cfb-bb5c-5f53b8cceffe')]}

## Intent

In [145]:
from typing import TypedDict, List, Literal
from langchain_core.messages import BaseMessage, SystemMessage
from pydantic import BaseModel
from langchain_core.output_parsers import PydanticOutputParser

llm = ChatOllama(
    model="deepseek-r1:14B",
    temperature=0,
)

class Rooms(BaseModel):
    rooms: Literal["office", "bedroom"]
    topics: Literal["home/lights/office", "home/lights/bedroom"]
    status: Literal["off", "on"]

class State(TypedDict):
    messages: List[BaseMessage]
    intent: str | None
    policy_check: bool
    rooms: Rooms | None

config = {
    "bedroom": "home/lights/bedroom",
    "office": "home/lights/office"
}

INTENT_PROMPT = SystemMessage(
    content=(
        "Classify the user's intent.\n\n"
        "Return exactly ONE of the following strings:\n"
        "- search\n"
        "- read\n"
        "- control\n"
        "- chat\n\n"
        "Return ONLY the label."
    )
)
# PARAMETER_EXTRACT_PROMPT = SystemMessage(
#     content = "Based on human message, refer which room's lights needs to be controlled and what needs to be done."
#     f"The docstring: \n {control_node.__doc__}"
#     "Also look at the Rooms pydantic object because you need to fill it in correctly:"
#     f"{Rooms.schema_json()}"
# )

parser = PydanticOutputParser(pydantic_object=Rooms)

# TODO: embed config in this prompt as well
CONTROL_PROMPT = SystemMessage(
    content = (
        "Based on human message, refer which room's lights needs to be controlled and what needs to be done."
        "You must output JSON that matches this schema:"
        f"{parser.get_format_instructions()}"
        "Do not include any text outside the JSON."
    )
)


def classify_intent(state: State):
    response = llm.invoke(
        [INTENT_PROMPT] + state["messages"]
    )

    intent = response.content.strip().lower()

    return {
        "intent": intent,
        "messages": state["messages"] + [response],
    }


def policy_check_control(state: State):
    policy_check = True

    return {
        "policy_check": policy_check,
        "messages": state["messages"]
    }


def classify_control(state: State):
    response = llm.invoke(
        [CONTROL_PROMPT] + state["messages"]
    )
    rooms = response.content.strip().lower()
    print(rooms)
    rooms: Rooms = parser.parse(response.content)
    return {
        "rooms": rooms,
        "messages": state["messages"] + [response]
    }



def route_by_intent(state: State) -> str:
    intent = state.get("intent")

    if intent == "read":
        return "read_node"
    if intent == "control":
        return "control_node"

    return "chat_node"


def read_node(state: State):
    print("→ READ")
    return state

def control_parameter_node(state: State):
    '''Sends commands for turning on or off lights.
    Based on `topics` field in `rooms` parameter, it will decide which mqtt topic
    it will send the message. `status` field in `rooms` class is needed for `on` 
    or `off` command.

    Args:
        state (State): the state has been being managed by Langgraph.
        rooms (Rooms): Rooms class to be parsed in the function to send 
        necessary messages to proper mqtt topic.

    Returns:
        state (State): the state has been being managed by Langgraph.
    '''
    response = llm.invoke(
        [CONTROL_PROMPT] + state["messages"]
    )
    rooms = response.content.strip().lower()
    print(rooms)
    rooms: Rooms = parser.parse(response.content)
    return {
        "rooms": rooms,
        "messages": state["messages"] + [response]
    }

def chat_node(state: State):
    print("→ CHAT")
    return state

/var/folders/7j/qr6tvgr97s3b65k1tx46fry80000gn/T/ipykernel_21765/1876980877.py:6: ResourceWarning: unclosed <socket.socket fd=90, family=2, type=1, proto=6, laddr=('127.0.0.1', 60424), raddr=('127.0.0.1', 11434)>
  llm = ChatOllama(


In [146]:
from langgraph.graph import StateGraph, START, END

graph = StateGraph(State)

graph.add_edge(START, "intent")
graph.add_node("intent", classify_intent)
graph.add_node("read_node", read_node)
graph.add_node("control_parameter_node", control_parameter_node)
graph.add_node("chat_node", chat_node)

graph.add_conditional_edges(
    "intent",
    route_by_intent,
    {
        "read_node": "read_node",
        "control_node": "control_parameter_node",
        "chat_node": "chat_node",
    },
)

graph.add_edge("read_node", END)
graph.add_edge("control_parameter_node", END)
graph.add_edge("chat_node", END)

app = graph.compile()


In [147]:
from langchain_core.messages import HumanMessage

result = app.invoke({
    "messages": [HumanMessage(content="Turn off the bedroom lights")],
    "intent": None,
    "rooms": None
})

the office light to turn on
okay, so i need to figure out how to respond to this user's message about controlling the lights in different rooms. the user provided a specific schema that i have to follow strictly. let me break it down.

first, the user sent two messages: "turn off the bedroom lights" and "control the office light to turn on." each of these needs to be converted into json according to the given schema.

looking at the schema, each json object should have three keys: rooms, topics, and status. the rooms can be either "office" or "bedroom," topics are specific strings like "home/lights/office," and status is either "off" or "on."

for the first message, "turn off the bedroom lights":
- the room is clearly the bedroom.
- the topic should match the bedroom, so it's "home/lights/bedroom."
- the action is to turn off, so status is "off."

putting that together, the json would be {"rooms": "bedroom", "topics": "home/lights/bedroom", "status": "off"}.

for the second message, "c

In [148]:
result

{'messages': [HumanMessage(content='Turn off the bedroom lights', additional_kwargs={}, response_metadata={}),
  AIMessage(content='control', additional_kwargs={}, response_metadata={'model': 'deepseek-r1:14B', 'created_at': '2026-01-08T01:00:06.791113Z', 'done': True, 'done_reason': 'stop', 'total_duration': 22920448083, 'load_duration': 76085000, 'prompt_eval_count': 40, 'prompt_eval_duration': 1634748791, 'eval_count': 204, 'eval_duration': 19829456701, 'logprobs': None, 'model_name': 'deepseek-r1:14B', 'model_provider': 'ollama'}, id='lc_run--019b9b1d-db7c-7661-83af-c12b821698cb-0', usage_metadata={'input_tokens': 40, 'output_tokens': 204, 'total_tokens': 244}),
  AIMessage(content=' the office light to turn on\nOkay, so I need to figure out how to respond to this user\'s message about controlling the lights in different rooms. The user provided a specific schema that I have to follow strictly. Let me break it down.\n\nFirst, the user sent two messages: "Turn off the bedroom lights

In [149]:
result.keys()

dict_keys(['messages', 'intent', 'rooms'])